# Hashing HXXH

In [3]:
import qiskit

%cd -q ../..

from rivet_transpiler import get_circuit_hash
from rivet_transpiler import transpile

from qiskit_ibm_runtime.fake_provider import FakeLimaV2

In [6]:
backend = FakeLimaV2() 

# qiskit.visualization.plot_gate_map(backend)

In [8]:
hx_circuit = qiskit.QuantumCircuit(2)
hx_circuit.h(1)
hx_circuit.x(1)

print("HX circuit:")
print(hx_circuit.draw())
print()

hash_hx_circuit = get_circuit_hash(hx_circuit)

hash_hx_circuit

HX circuit:
               
q_0: ──────────
     ┌───┐┌───┐
q_1: ┤ H ├┤ X ├
     └───┘└───┘



8895418907420795653914132157143967161278273931489570184451554769513788291599

In [9]:
xh_circuit = qiskit.QuantumCircuit(2)
xh_circuit.x(1)
xh_circuit.h(1)

print("XH circuit:")
print(xh_circuit.draw())
print()

hash_xh_circuit = get_circuit_hash(xh_circuit)

hash_xh_circuit

XH circuit:
               
q_0: ──────────
     ┌───┐┌───┐
q_1: ┤ X ├┤ H ├
     └───┘└───┘



13745032090551206655011846936491227366863944402211228876193755914927240176279

In [11]:
if hash_hx_circuit == hash_xh_circuit:
    print("The circuits have the same hash value.")
else:
    print("The circuits have different hash values.")

print(get_circuit_hash(hx_circuit))
print(get_circuit_hash(xh_circuit))

The circuits have different hash values.
8895418907420795653914132157143967161278273931489570184451554769513788291599
13745032090551206655011846936491227366863944402211228876193755914927240176279


In [7]:
def bind_parameters_with_offset(circuit, offset=0):

    bound_circuit = circuit.copy()

    for index, parameter in enumerate(bound_circuit.parameters):

        bound_circuit.assign_parameters(
            {parameter: index + offset},
            inplace=True)

    return bound_circuit

In [8]:
# Define Bound Circuits
bound_circuit = bind_parameters_with_offset(litmus_circuit, offset=0)
bound_circuit_same_parameters = bind_parameters_with_offset(litmus_circuit, offset=0)
bound_circuit_other_parameters = bind_parameters_with_offset(litmus_circuit, offset=1)

In [9]:
# Display Circuits
display(bound_circuit.draw(fold=-1))
display(bound_circuit_same_parameters.draw(fold=-1))
display(bound_circuit_other_parameters.draw(fold=-1))

┌───────┐          ┌───┐ Litmus 
Litmus_0_0: ┤ Rz(0) ├──■───────┤ X ├───░────
            ├───────┤┌─┴─┐     └─┬─┘   ░    
Litmus_0_1: ┤ Rz(1) ├┤ X ├──■────┼─────░────
            ├───────┤└───┘┌─┴─┐  │     ░    
Litmus_0_2: ┤ Rz(2) ├─────┤ X ├──■─────░────
            └───────┘     └───┘        ░

┌───────┐          ┌───┐ Litmus 
Litmus_0_0: ┤ Rz(0) ├──■───────┤ X ├───░────
            ├───────┤┌─┴─┐     └─┬─┘   ░    
Litmus_0_1: ┤ Rz(1) ├┤ X ├──■────┼─────░────
            ├───────┤└───┘┌─┴─┐  │     ░    
Litmus_0_2: ┤ Rz(2) ├─────┤ X ├──■─────░────
            └───────┘     └───┘        ░

┌───────┐          ┌───┐ Litmus 
Litmus_0_0: ┤ Rz(1) ├──■───────┤ X ├───░────
            ├───────┤┌─┴─┐     └─┬─┘   ░    
Litmus_0_1: ┤ Rz(2) ├┤ X ├──■────┼─────░────
            ├───────┤└───┘┌─┴─┐  │     ░    
Litmus_0_2: ┤ Rz(3) ├─────┤ X ├──■─────░────
            └───────┘     └───┘        ░

In [10]:
# Display Hashes
display(get_circuit_hash(bound_circuit))
display(get_circuit_hash(bound_circuit_same_parameters))
display(get_circuit_hash(bound_circuit_other_parameters))

64409077824029668028239311688647418237442796507979302561569167897928616433286

64409077824029668028239311688647418237442796507979302561569167897928616433286

49146857853148014873065169400610474609017800429749393590319342929057395408014

In [11]:
# Define 3 circuits
#for each circuit different properties can be added. For example:
    # optimization_level=3,
    # initial_layout=[1, 2, 3]

transpiled_litmus_circuit = transpile(
    litmus_circuit,
    backend,
    seed_transpiler=1234,
)

transpiled_litmus_circuit_same_seed = transpile(
    litmus_circuit,
    backend,
    seed_transpiler=1234,
)

transpiled_litmus_circuit_other_seed = transpile(
    litmus_circuit,
    backend,
    seed_transpiler=777,
)

In [12]:
# Display Circuits

display(transpiled_litmus_circuit.draw(fold=-1))
display(transpiled_litmus_circuit_same_seed.draw(fold=-1))
display(transpiled_litmus_circuit_other_seed.draw(fold=-1))

┌──────────────┐          ┌───┐          ┌───┐ Litmus 
Litmus_0_2 -> 0 ┤ Rz(Litmus_2) ├───────■──┤ X ├──■───────┤ X ├───░────
                ├──────────────┤     ┌─┴─┐└─┬─┘┌─┴─┐┌───┐└─┬─┘   ░    
Litmus_0_0 -> 1 ┤ Rz(Litmus_0) ├──■──┤ X ├──■──┤ X ├┤ X ├──■─────░────
                ├──────────────┤┌─┴─┐└───┘     └───┘└─┬─┘        ░    
Litmus_0_1 -> 2 ┤ Rz(Litmus_1) ├┤ X ├─────────────────■──────────░────
                └──────────────┘└───┘                            ░    
 ancilla_0 -> 3 ──────────────────────────────────────────────────────
                                                                      
 ancilla_1 -> 4 ──────────────────────────────────────────────────────

┌──────────────┐          ┌───┐          ┌───┐ Litmus 
Litmus_0_2 -> 0 ┤ Rz(Litmus_2) ├───────■──┤ X ├──■───────┤ X ├───░────
                ├──────────────┤     ┌─┴─┐└─┬─┘┌─┴─┐┌───┐└─┬─┘   ░    
Litmus_0_0 -> 1 ┤ Rz(Litmus_0) ├──■──┤ X ├──■──┤ X ├┤ X ├──■─────░────
                ├──────────────┤┌─┴─┐└───┘     └───┘└─┬─┘        ░    
Litmus_0_1 -> 2 ┤ Rz(Litmus_1) ├┤ X ├─────────────────■──────────░────
                └──────────────┘└───┘                            ░    
 ancilla_0 -> 3 ──────────────────────────────────────────────────────
                                                                      
 ancilla_1 -> 4 ──────────────────────────────────────────────────────

┌──────────────┐     ┌───┐                       ░    
Litmus_0_2 -> 0 ┤ Rz(Litmus_2) ├─────┤ X ├─────────────────■─────░────
                └──────────────┘     └─┬─┘                 │     ░    
                ┌──────────────┐┌───┐  │  ┌───┐     ┌───┐┌─┴─┐ Litmus 
Litmus_0_1 -> 1 ┤ Rz(Litmus_1) ├┤ X ├──■──┤ X ├──■──┤ X ├┤ X ├───░────
                ├──────────────┤└─┬─┘     └─┬─┘┌─┴─┐└─┬─┘└───┘   ░    
Litmus_0_0 -> 2 ┤ Rz(Litmus_0) ├──■─────────■──┤ X ├──■──────────░────
                └──────────────┘               └───┘             ░    
 ancilla_0 -> 3 ──────────────────────────────────────────────────────
                                                                      
 ancilla_1 -> 4 ──────────────────────────────────────────────────────

In [13]:
# Display Hashes

display(get_circuit_hash(transpiled_litmus_circuit))
display(get_circuit_hash(transpiled_litmus_circuit_same_seed))
display(get_circuit_hash(transpiled_litmus_circuit_other_seed))

53917699254062553287735835084446225408834725910758070812350368677283212102985

53917699254062553287735835084446225408834725910758070812350368677283212102985

58942053007377106266998516815207512147048211035462556228034425281922438593770

In [14]:
QUBITS_COUNT = 100
MAX_LAYERS_COUNT = 5

GATES = ['x', 'y', 'z',
         'rx', 'ry', 'rz',
         'rxx', 'ryy', 'rzz',
         'swap', 'id']

gates_counts = []
layers_counts = []
elapsed_times = []

for layers_count in range(MAX_LAYERS_COUNT):

    circuit = qiskit.circuit.library.EfficientSU2(QUBITS_COUNT,
                                                  reps=layers_count,
                                                  su2_gates=GATES,
                                                  entanglement="circular",
                                                  skip_final_rotation_layer=True)


    circuit_decomposed = circuit.decompose(None, 1)
    gates_count = len(circuit_decomposed.data)
    print("gates_count:", gates_count)

    start_time = time()
    circuit_hash = get_circuit_hash(circuit_decomposed)
    elapsed_time = time() - start_time
    gates_counts.append(gates_count)
    layers_counts.append(layers_count)
    elapsed_times.append(elapsed_time)
    print(f"elapsed_time:{elapsed_time:.5f}")

    # circuit_decomposed.draw(fold=-1)

gates_count: 0
elapsed_time:0.00023
gates_count: 1000
elapsed_time:0.11667
gates_count: 2000
elapsed_time:0.23836
gates_count: 3000
elapsed_time:0.43414
gates_count: 4000
elapsed_time:0.57078


From the plot  for the hashing time as a function of gate counts **a linear dependency is observed**:

In [15]:
#Plot Hashing Time as a function of the number of gates

# plt.title("Hashing Time")

# plt.xlabel("Gates count")
# plt.ylabel("Time, seconds")

# plt.plot(gates_counts, elapsed_times);

![image.png](Fig3_timehashing.png) 

*Fig.3. Hashing time as a function of gate counts for the circuit above.*